# Extraction of Relevant Action and Object Knowledge from the Web

- Definition of Action and Object Knowledge
- Extraction of Action Knowledge:
    - choosing a starting action ("Cutting")
    - gathering synonyms and hyponyms from WordNet, VerbNet and Thesaurus
    - filtering the gathered verbs using Recipe1M+ and WikiHow
- Extraction of Object Knowledge:
    - choosing an object group ("Fruits & Vegetables")
    - gathering all fruits and vegetables from the FoodOn
    - filtering the gathered fruits and vegetables using Recipe1M+
    - gathering task-specific information using 3 different embeddings, 2 LLMs (ChatGPT & GPT-4) as well as 2 Recipe1M+ filtering methods

## Extraction of Action Knowledge
bla bla bla

In [ ]:
# imports
import pywsd.lesk as lesk
import pandas as pd
from nltk.corpus import verbnet, wordnet

### Extracting Synonyms and Hyponyms from WordNet & VerbNet

In [ ]:
# setting the target action and an exemplary sentence
target_action = "cut"
example_sent = "Cut the apple into 2 halves"

# extracting the synset for the given target action
synset = lesk.cosine_lesk(example_sent, target_action, pos='v')
print(f"Synset: {synset}")
synset = wordnet.synset('cut.v.01')

# gathering all synonyms and hyponyms from WordNet
verbs = synset.lemma_names()
for h in synset.hyponyms():
    verbs.extend(h.lemma_names())
    

# gathering synonyms from VerbNet
key = str(synset.lemmas()[0].key()).replace("::", "")
print(f'VerbNet Key: {key}')
class_name = verbnet.classids(wordnetid='key')
if not class_name:
    classes = verbnet.classids(lemma=target_action)
    for c in classes:
        front = c.split('-')[0]
        if str(front) == target_action:
            class_name = c
            # print(f'ClassName={class_name}')
vn_class = verbnet.fileids(class_name)[0]
verbs.extend(verbnet.lemmas(vnclass=vn_class))
verbs = set(verbs)
print(f"{len(verbs)} synonyms or hyponyms found")

In [ ]:
# pre-process the found synonyms and hyponyms
filtered_verbs = {v for v in verbs if "_" not in v}
filtered_verbs = sorted(set(filtered_verbs))

print(f"{len(filtered_verbs)} remaining words:")
for verb in filtered_verbs:
    print(verb)

### Filtering the extracted verbs

In [ ]:
v_occurrences = "./verb_occurrences.csv"

# read and map the (extracted) occurrence data
voc_dat = pd.read_csv(v_occurrences)
voc_dat = voc_dat.astype({'Title':'int','Title Desc':'int','Method':'int','Step Headline':'int','Step Desc':'int','SUM':'int'})

# remove all verbs with 0 occurrences
most_used = voc_dat[(voc_dat['SUM'] > 0)]
print(f"{len(most_used)} verbs that occur at least 1x")

# remove all verbs with too few available sentences (Step Desc <= threshold)
thresh = 50
most_used = most_used[(most_used['Step Desc'] >= thresh)]
print(f"{len(most_used)} verbs that cover more than {thresh} step descriptions:")
print(most_used)

## not fitting: DROP, RIP, SHAVE, TAP
## missing but important: FILLET, HASH, HALVE, MINCE, QUARTER, SLIVER

## Extraction of Object Knowledge
bla bla bla

In [ ]:
# imports
from rdflib import Graph, Literal, Namespace, RDF, RDFS, URIRef
from rdflib.plugins.sparql import prepareQuery
#from typing import List
import pandas as pd
import gensim
import torch
import torchtext

### Extracting fruits and vegetables from the FoodOn

In [ ]:
# load the ontology and set the namespace prefixes
foodon_loc = "./foodon.owl"
g = Graph()
g.parse(foodon_loc)

FOOD = Namespace("http://purl.obolibrary.org/obo/")
RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")

# get the fruit data through the SPARQL query 
query = prepareQuery(   
    """
    SELECT ?fruit_label (SAMPLE(?fruit_id) AS ?rndm_fruit_id) (SAMPLE(?def) AS ?rndm_def)
    WHERE {
        ?fruit_id rdfs:label ?label.
        ?fruit_id rdfs:subClassOf+ food:PO_0009001.
        OPTIONAL { ?fruit_id food:IAO_0000115 ?def. }

        BIND (LCASE(STR(?label)) AS ?str_label).
        BIND (STRBEFORE(?str_label, "(") AS ?fruit_label).
        FILTER CONTAINS(?str_label, "whole").
        FILTER NOT EXISTS { ?fruit_id rdfs:subClassOf* food:PO_0030104. }
        FILTER (?fruit_id != food:FOODON_03304644).
    }
    GROUP BY ?fruit_label
    ORDER BY ?fruit_label
    """,
    initNs={"food": FOOD, "rdfs": RDFS}
)
fruits = g.query(query)

# get the vegetable data through the SPARQL query 
query = prepareQuery(
    """
    SELECT ?veg_label (SAMPLE(?veg_id) AS ?rndm_veg_id) (SAMPLE(?def) AS ?rndm_def)
    WHERE {
        ?veg_id rdfs:label ?label.
        ?veg_id rdfs:subClassOf+ food:FOODON_03302008.
        OPTIONAL { ?veg_id food:IAO_0000115 ?def. }

        BIND (LCASE(STR(?label)) AS ?str_label).
        BIND (STRBEFORE(?str_label, "(") AS ?veg_label).
        FILTER NOT EXISTS { ?veg_id rdfs:subClassOf* food:FOODON_03302007. }
    }
    GROUP BY ?veg_label
    ORDER BY ?veg_label
    """,
    initNs={"food": FOOD, "rdfs": RDFS}
)
veggies = g.query(query)

# convert query results into panda dataframes for further analysis
fruit_list = [(str(row[0]), str(row[1]), str(row[2])) for row in fruits]
veggie_list = [(str(row[0]), str(row[1]), str(row[2])) for row in veggies]

fruit_df = pd.DataFrame(fruit_list, columns=["label", "rndm_id", "rndm_def"])
veggie_df = pd.DataFrame(veggie_list, columns=["label", "rndm_id", "rndm_def"])
combined_df = pd.concat([fruit_df, veggie_df], ignore_index=True)
print(combined_df)

### Filter the fruits and vegetables using WikiHow and Recipe1M+ data

In [ ]:
f_occurrences = "./fruit_occurrences.csv"

# read and map the (extracted) occurrence data
foc_dat = pd.read_csv(f_occurrences)
foc_dat = foc_dat.astype({'Recipes-Title':'int','Recipes-Title [%]':'float', 'Recipes-Steps':'int','Recipes-Steps [%]':'float',
                'WikiHow-Title':'int','WikiHow-Title [%]':'float', 'WikiHow-TitleDescription':'int','WikiHow-TitleDescription [%]':'float',
                'WikiHow-Method':'int','WikiHow-Method [%]':'float', 'WikiHow-StepHeadline':'int','WikiHow-StepHeadline [%]':'float',
                'WikiHow-StepDescription':'int','WikiHow-StepDescription [%]':'float'})

# remove all items with too few occurrences in any column (less than 1%)
thresh = 0.01
most_used = foc_dat[(foc_dat['Recipes-Title [%]'] >= thresh) | (foc_dat['Recipes-Steps [%]'] >= thresh) | (foc_dat['WikiHow-Title [%]'] >= thresh) | 
                  (foc_dat['WikiHow-TitleDescription [%]'] >= thresh) | (foc_dat['WikiHow-Method [%]'] >= thresh) | (foc_dat['WikiHow-StepHeadline [%]'] >= thresh) |
                  (foc_dat['WikiHow-StepDescription [%]'] >= thresh)]
print(most_used)

### Extract object properties using embeddings

In [ ]:
# possible food parts and their connection
parts = ['core', 'shell', 'peel', 'stem']
foods = list(most_used['Name'])
food_parts = {
    'almond': ['shell'],
    'apple': ['core', 'peel'],
    'banana': ['peel'],
    'cherry': ['core'],
    'citron': ['peel'],
    'coconut': ['shell'],
    'cucumber': ['peel', 'stem'],
    'kumquat': ['peel'],
    'lemon': ['peel'],
    'lime': ['peel'],
    'olive': ['core'],
    'orange': ['peel'],
    'pepper': ['stem'],
    'pineapple': ['core', 'peel'],
    'pumpkin': ['core', 'peel'],
    'squash': ['core', 'peel'],
    'strawberry': ['stem'],
    'tomato': ['peel', 'stem']
}

# define function for finding the key based on the given concept name
def find_key(concept):
    concept_is_synset = "bn:" in concept
    keys = [key for key in nasari.index_to_key if concept in key.lower()]
    for key in keys:
        cut = key.split('__')[1].lower()
        if (cut == concept and not concept_is_synset) or (concept_is_synset and concept in key.lower()):
            return key
    return concept

# load ConceptNet Numberbatch & NASARI embeddings
numberbatch = gensim.models.KeyedVectors.load_word2vec_format('./numberbatch-en.txt', binary=False)
nasari = gensim.models.KeyedVectors.load_word2vec_format('./NASARI_embed_english.txt', binary=False)

# cosine similarity between GloVe embeddings
glove = torchtext.vocab.GloVe(name="6B", dim=50)

for f in foods:
    for p in parts:
        sim = torch.cosine_similarity(glove[f].unsqueeze(0), glove[p].unsqueeze(0)).item()
        if sim >= 0.5:
            print(f'[GloVe] Similarity for {p} in {f}: {sim}')

# cosine similarity between ConceptNet Numberbatch embeddings
for f in foods:
    for p in parts:
        sim = numberbatch.similarity(f, p)
        if sim >= 0.3:
            print(f'[CN Numberbatch] Similarity for {p} in {f}: {sim}')
            
# cosine similarity between NASARI embeddings
# Sadly, the BabelNet synsets for core (bn:04772260n) does not exist in the NASARI embeddings and 
# for 'shell' we need to look for the concrete synset (bn:00071005n) instead 
parts_nasari = parts = ['bn:00071005n', 'peel_(fruit)', 'plant_stem']
for f in foods:
    for p in parts_nasari:
        f_key = find_key(f)
        p_key = find_key(p)
        if (f_key in nasari.index_to_key) and (p_key in nasari.index_to_key):
            sim = nasari.similarity(f_key, p_key)
            if sim >= 0.75:
                print(f'[NASARI] Similarity for {p} and {f}: {sim}')            

### Extract object properties using Recipe1M+ data

In [ ]:
## ToDo